In [8]:
import pandas as pd
import numpy as np

The labels are parsed to grid style

In [21]:
import os

def coordinates_to_grid(coordinates: pd.DataFrame, i_dim=(1280, 720), o_dim=(768, 432), grid=(48, 27)):
    grid_size = (o_dim[0] / grid[0], o_dim[1] / grid[1])

    # Convert dimensions
    coordinates.loc[:, 'x-coordinate'] = coordinates['x-coordinate'].div(i_dim[0]).mul(o_dim[0]).round(0)
    coordinates.loc[:, 'y-coordinate'] = coordinates['y-coordinate'].div(i_dim[1]).mul(o_dim[1]).round(0)

    # Find grids
    coordinates['x-grid'] = (coordinates['x-coordinate'].div(grid_size[0]))
    coordinates['y-grid'] = (coordinates['y-coordinate'].div(grid_size[1]))

    # Find grid offsets (the x, y coordinates within the grid)
    coordinates['x-grid-offset'] = coordinates['x-grid'].mod(1)
    coordinates['y-grid-offset'] = coordinates['y-grid'].mod(1)

    # Round the grid coordinates
    coordinates['x-grid'] = coordinates['x-grid'].floordiv(1).astype(int)
    coordinates['y-grid'] = coordinates['y-grid'].floordiv(1).astype(int)

    return coordinates

def get_clips_dirs():
    dirs = []

    for dir in os.walk('dataset'):
        if (not 'Clip' in dir[0]): continue
        dirs.append(dir[0])

    return dirs

def load_and_transform_clip_labels(clip_dir):
    labels = pd.read_csv(os.path.join(clip_dir, 'Label.csv'))
    grids = coordinates_to_grid(labels.loc[:, ['x-coordinate', 'y-coordinate']])
    grids = grids.merge(labels[['status', 'file name', 'visibility']], left_index=True, right_index=True, how='left')
    return grids

In [22]:
import PIL.Image

def create_instances(clip_dir, files):
    instances = []

    # 2 frames overlap (i.e. step = 3)
    for i in range(0, len(files), 3):
        if (len(files) - i) < 5: break
        frames = []

        for j in range(5):
            if (i + j) >= len(files): break
            path = os.path.join(clip_dir, files[i + j])

            image = PIL.Image.open(path).convert('RGB').resize((768, 432))
            image = np.array(image).transpose((2, 0, 1))  # Convert to (3, 432, 768)
            image = image / 255.0 # Normalize to [0, 1]
            frames.append(image)

        instances.append(np.array(frames).reshape(5 * 3, 432, 768)) # Shape: (15, 432, 768)
    return np.array(instances)

In [23]:
def create_labels(labels):
    instances = []

    for i in range(0, len(labels), 3):
        if (len(labels) - i) < 5: break
        frames = []

        for j in range(5):
            if (i + j) >= len(labels): break
            grid_confidence = np.zeros((27, 48))
            grid_confidence[labels['y-grid'][i + j], labels['x-grid'][i + j]] = 1.0
            
            grid_x_offset = np.zeros((27, 48))
            grid_x_offset[labels['y-grid'][i + j], labels['x-grid'][i + j]] = labels['x-grid-offset'][i + j]

            grid_y_offset = np.zeros((27, 48))
            grid_y_offset[labels['y-grid'][i + j], labels['x-grid'][i + j]] = labels['y-grid-offset'][i + j]

            frames.append(np.concatenate((grid_confidence.reshape(1, 27, 48), grid_x_offset.reshape(1, 27, 48), grid_y_offset.reshape(1, 27, 48)), axis=0).transpose(1, 2, 0))

        instances.append(np.array(frames))
    return np.array(instances)

In [30]:
import h5py
import tqdm

clip_dirs = get_clips_dirs()
batch_size = 50
file_counter = 0
rest_labels = None
rest_instances = None

for clip_dir in tqdm.tqdm(clip_dirs):
    name = clip_dir.replace('/', '_')

    try:
        loaded_labels = load_and_transform_clip_labels(clip_dir)
        clip_instances = create_instances(clip_dir, loaded_labels['file name'].values)
        clip_labels = create_labels(loaded_labels)

        if (rest_labels is not None):
            clip_labels = np.concatenate((rest_labels, clip_labels), axis=0)
            clip_instances = np.concatenate((rest_instances, clip_instances), axis=0)
        
        total_instances = len(clip_instances)

        for start_idx in range(0, total_instances, batch_size):
            end_idx = min(start_idx + batch_size, total_instances)

            if (end_idx - start_idx) < batch_size:
                rest_instances = clip_instances[start_idx:end_idx]
                rest_labels = clip_labels[start_idx:end_idx]
                break
            
            # Create a new file for each batch
            batch_filename = f'compiled_dataset/{file_counter:03d}.hdf5'
            file_counter += 1
            
            with h5py.File(batch_filename, "w") as file:
                file.create_dataset('instances', data=clip_instances[start_idx:end_idx])
                file.create_dataset('labels', data=clip_labels[start_idx:end_idx])

    except:
        print(f'Failed to process {clip_dir} ({name})')
        continue

  4%|▍         | 4/95 [00:23<08:28,  5.59s/it]

Failed to process dataset/game5/Clip7 (dataset_game5_Clip7)
Failed to process dataset/game5/Clip15 (dataset_game5_Clip15)
Failed to process dataset/game5/Clip12 (dataset_game5_Clip12)


 13%|█▎        | 12/95 [00:39<05:23,  3.89s/it]

Failed to process dataset/game5/Clip3 (dataset_game5_Clip3)


 16%|█▌        | 15/95 [00:43<03:24,  2.56s/it]

...

Failed to process dataset/game9/Clip5 (dataset_game9_Clip5)


 91%|█████████ | 86/95 [09:57<01:22,  9.21s/it]

Failed to process dataset/game7/Clip6 (dataset_game7_Clip6)
Failed to process dataset/game7/Clip1 (dataset_game7_Clip1)
Failed to process dataset/game7/Clip8 (dataset_game7_Clip8)
Failed to process dataset/game7/Clip9 (dataset_game7_Clip9)
Failed to process dataset/game7/Clip7 (dataset_game7_Clip7)


100%|██████████| 95/95 [10:09<00:00,  6.41s/it]

Failed to process dataset/game7/Clip5 (dataset_game7_Clip5)
Failed to process dataset/game7/Clip4 (dataset_game7_Clip4)
Failed to process dataset/game7/Clip3 (dataset_game7_Clip3)
